In [ ]:
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16

from matplotlib.pyplot import imread,imshow
from skimage.transform import rescale, resize
from skimage.io import imsave

import os
import numpy as np
from keras.models import load_model
from keras.models import load_model

In [ ]:
whitelist_emb_file = 'whitelist_emb.npy' #precomputed embeddings 
phish_emb_file = 'phishing_emb.npy' #precomputed embeddings 
output_dir = './'
saved_model_name = 'model' 

# Dataset parameters 
dataset_path = '../../datasets/WhitePhish/'
reshape_size = [224,224,3]
num_targets = 155 

#adv parameters
epsilon = 0.005
batch_size = 3
trials = 5

# Load dataset 

In [ ]:

def read_imgs_per_website(data_path,targets,imgs_num,reshape_size,start_target_count):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    
    all_file_names = []
    targets_list = targets.splitlines()
    count = 0
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        print(target_path)
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            try:
                img = imread(target_path+'/'+file_names[j])
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                all_labels[count,:] = i + start_target_count
                all_file_names.append(file_names[j])
                count = count + 1
            except:
                #some images were saved with a wrong extensions 
                try:
                    img = imread(target_path+'/'+file_names[j],format='jpeg')
                    img = img[:,:,0:3]
                    all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                    all_labels[count,:] = i + start_target_count
                    all_file_names.append(file_names[j])
                    count = count + 1
                except:
                    print('failed at:')
                    print('***')
                    print(file_names[j])
                    break 
    return all_imgs,all_labels,all_file_names



In [ ]:
# Read images legit (train)
data_path = dataset_path + 'legitimate_whitelist/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 9363
all_imgs_train,all_labels_train,all_file_names_train = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

# Read images phishing
data_path = dataset_path + 'phishing/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 1195
all_imgs_test,all_labels_test,all_file_names_test = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

In [ ]:
X_train_legit = all_imgs_train
y_train_legit = all_labels_train


phish_train_idx = np.load(output_dir+'train_idx.npy')
phish_test_idx = np.load(output_dir+'test_idx.npy')


X_train_phish = all_imgs_test[phish_train_idx,:]
y_train_phish = all_labels_test[phish_train_idx,:]

X_test_phish = all_imgs_test[phish_test_idx,:]
y_test_phish = all_labels_test[phish_test_idx,:]
y_test_phish_non_ordered = all_labels_test[phish_test_idx,:]


def order_random_array(orig_arr,y_orig_arr,targets):
    sorted_arr = np.zeros(orig_arr.shape)
    y_sorted_arr = np.zeros(y_orig_arr.shape)
    count = 0
    for i in range(0,targets):
        for j in range(0,orig_arr.shape[0]):
            if y_orig_arr[j] == i:
                sorted_arr[count,:,] = orig_arr[j,:]
                y_sorted_arr[count,:] = i
                count = count + 1
    return sorted_arr,y_sorted_arr 
X_train_phish_features,y_train_phish_ordered = order_random_array(X_train_phish_features,y_train_phish,num_targets)

X_test_phish,y_test_phish = order_random_array(X_test_phish,y_test_phish,num_targets)
X_train_phish,y_train_phish = order_random_array(X_train_phish,y_train_phish,num_targets)


#get start and end of each label
def start_end_each_target_not_complete(num_target,labels):
    prev_target = labels[0]
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = labels[0]
    if not labels[0] == 0:
        start_end_each_target[0,0] = -1
        start_end_each_target[0,1] = -1
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[int(labels[i-1]),1] = int(i-1)
            #count_target = count_target + 1
            start_end_each_target[int(labels[i]),0] = int(i)
            prev_target = labels[i]
    start_end_each_target[int(labels[-1]),1] = int(labels.shape[0]-1)
    
    for i in range(1,num_target):
        if start_end_each_target[i,0] == 0:
            start_end_each_target[i,0] = -1
            start_end_each_target[i,1] = -1
    return start_end_each_target

labels_start_end_train_phish = start_end_each_target_not_complete(num_targets,y_train_phish)
labels_start_end_test_phish = start_end_each_target_not_complete(num_targets,y_test_phish)

def start_end_each_target(num_target,labels):
    prev_target = 0
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = 0
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[count_target,1] = i-1
            count_target = count_target + 1
            start_end_each_target[count_target,0] = i
            prev_target = prev_target + 1
    start_end_each_target[num_target-1,1] = labels.shape[0]-1
    return start_end_each_target

labels_start_end_train_legit = start_end_each_target(num_targets,y_train_legit)


# Load Model 

In [ ]:
from keras.models import load_model
margin = 2.2
def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value

full_model = load_model(output_dir+saved_model_name+'.h5', custom_objects={'loss': loss})
inside_model = full_model.layers[3]

#define custom_loss
def custom_loss(margin):
    def loss(y_true,y_pred):
        loss_value = K.maximum(y_true, margin + y_pred)
        loss_value = K.mean(loss_value,axis=0)
        return loss_value
    return loss
my_loss = custom_loss(60) #set margin to a large value in order to always have a non-zero loss in adv generation.

#get tf session
sess = K.get_session()

# Load pretrained embeddings 

In [ ]:

X_train_legit_features = np.load(output_dir+whitelist_emb_file)
phish_features = np.load(output_dir+phish_emb_file)

X_test_phish_features = phish_features[phish_test_idx,:]
X_train_phish_features = phish_features[phish_train_idx,:]


# Triplet sampling 

In [ ]:
def pick_pos_img_idx(prob_phish,img_label):
    if np.random.uniform() > prob_phish:
        #take image from legit
        class_idx_start_end = labels_start_end_train_legit[img_label,:]
        same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
        img = X_train_legit[same_idx,:]
    else:
        #take from phish
        if not labels_start_end_train_phish[img_label,0] == -1:
            class_idx_start_end = labels_start_end_train_phish[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_phish[same_idx,:]
        else:
            class_idx_start_end = labels_start_end_train_legit[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_legit[same_idx,:]
    return img


def pick_neg_img(anchor_idx,num_targets):
    if anchor_idx == 0:
        targets = np.arange(1,num_targets)
    elif anchor_idx == num_targets -1:
        targets = np.arange(0,num_targets-1)
    else:
        targets = np.concatenate([np.arange(0,anchor_idx),np.arange(anchor_idx+1,num_targets)])
    diff_target_idx = np.random.randint(low = 0,high = num_targets-1)
    diff_target = targets[diff_target_idx]
    
    class_idx_start_end = labels_start_end_train_legit[diff_target,:]
    idx_from_diff_target = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
    img = X_train_legit[idx_from_diff_target,:]
    
    return img,diff_target


targets_file = open(data_path+'targets.txt', "r")
all_targets = targets_file.read()
all_targets = all_targets.splitlines()

def get_idx_of_target(target_name,all_targets):
    for i in range(0,len(all_targets)):
        if all_targets[i] == target_name:
            found_idx = i
            return found_idx

target_lists = [['microsoft','ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['apple','itunes','icloud'],['google','google_drive'],['alibaba','aliexpress']]

def get_associated_targets_idx(target_lists,all_targets):
    sub_target_lists_idx = []
    parents_ids = []
    for i in range(0,len(target_lists)):
        target_list = target_lists[i]
        parent_target = target_list[0]
        one_target_list = []
        parent_idx = get_idx_of_target(parent_target,all_targets)
        parents_ids.append(parent_idx)
        for child_target in target_list[1:]:
            child_idx = get_idx_of_target(child_target,all_targets)
            one_target_list.append(child_idx)
        sub_target_lists_idx.append(one_target_list)
    return parents_ids,sub_target_lists_idx 

parents_ids,sub_target_lists_idx  = get_associated_targets_idx(target_lists,all_targets)

def check_if_same_category(img_label1,img_label2):
    if_same = 0
    if img_label1 in parents_ids:
        if img_label2 in sub_target_lists_idx[parents_ids.index(img_label1)]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[0]:
        if img_label2 in sub_target_lists_idx[0] or img_label2 == parents_ids[0]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[1]:
        if img_label2 in sub_target_lists_idx[1] or img_label2 == parents_ids[1]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[2]:
        if img_label2 in sub_target_lists_idx[2] or img_label2 == parents_ids[2]:
            if_same = 1
    return if_same

# Find closest example to each query

In [ ]:
def compute_distance_pair(layer1,layer2):
    diff = layer1 - layer2
    l2_diff = np.mean(diff**2)
    return l2_diff

def argmax(lst):
    return lst.index(max(lst))

def argmin(lst):
    return lst.index(min(lst))


# Find closest example from the same website:
# Assume ordered training arrays (legit,train).

def find_closest_example(X_test_matrix,y_test_matrix):
    train_size = phish_train_idx.shape[0] + X_train_legit.shape[0]
    X_all_train = np.concatenate((X_train_phish_features,X_train_legit_features))
    pairwise_distance = np.zeros([X_test_matrix.shape[0],train_size])
    pairwise_distance_idx = np.zeros([X_test_matrix.shape[0],2])
    
    for i in range(0,X_test_matrix.shape[0]):
        pair1 = X_test_matrix[i,:]
        start_in_train = int(labels_start_end_train_legit[int(y_test_matrix[i]),0])
        end_in_train = int(labels_start_end_train_legit[int(y_test_matrix[i]),1])
        dist_list = []
        for j in range(start_in_train,end_in_train+1):
            pair2 = X_train_legit_features[j,:]
            l2_diff = compute_distance_pair(pair1,pair2)
            dist_list.append(l2_diff)
            
        min1 = min(dist_list)
        min1_idx = start_in_train + argmin(dist_list)
        
        dist_list2 = []
        start_in_phish_train = int(labels_start_end_train_phish[int(y_test_matrix[i]),0])
        end_in_phish_train = int(labels_start_end_train_phish[int(y_test_matrix[i]),1])
        
        min2 = -1
        if not labels_start_end_train_phish[int(y_test_matrix[i]),0] == -1:
            for j in range(start_in_phish_train,end_in_phish_train+1):
                pair2 = X_train_phish_features[j,:]
                l2_diff = compute_distance_pair(pair1,pair2)
                dist_list2.append(l2_diff)
            
            min2 = min(dist_list2)
            min2_idx = argmin(dist_list2) + start_in_phish_train
        
        if min1 < min2 or min2 == -1:
            pairwise_distance_idx[i,0] = min1_idx
        else:
            pairwise_distance_idx[i,0] = min2_idx
            #min is from phishing train
            pairwise_distance_idx[i,1] = 1
            
    return pairwise_distance_idx

pairwise_distance_idx = find_closest_example(X_test_phish_features,y_test_phish_non_ordered)

# Adv examples generation 

In [ ]:
def get_adv_example(triple,epsilon):
    
    # Initialize adversarial example 
    anchor_adv = np.zeros_like(triple[0])
    # Added noise
    anchor_noise = np.zeros_like(triple[0])

    y_true = tf.placeholder("float", [None,1])
    target = np.zeros([len(triple),1])
    target.astype(float)
    
    # Get the loss and gradient of the loss wrt the inputs
    loss_val = my_loss(y_true, full_model.output)
    grads = K.gradients(loss_val, full_model.input[0])
    
    # Get the sign of the gradient
    delta = K.sign(grads[0])
    
    dict_input = {y_true:target,full_model.input[0]:triple[0],full_model.input[1]:triple[1],full_model.input[2]:triple[2] }
    delta1 = sess.run(delta, feed_dict=dict_input)
    
    # Get noise
    anchor_noise = anchor_noise + delta1
    
    # Perturb the image
    anchor_adv = triple[0] + epsilon*delta1
    
    return anchor_noise,anchor_adv

In [ ]:
# Generate and save adv. examples for the phishing test set 
# For each query image, pick the closest example as the positive image
# Compute 5 trials (by randomly changing triplets)
# Save the embeddings for each trial 

X_test_phish_non_ordered = all_imgs_test[phish_test_idx,:]
y_test_phish_non_ordered = all_labels_test[phish_test_idx,:]

X_test_phish_adv = np.zeros_like(X_test_phish_non_ordered)

# initialize 3 empty arrays for the input image batch
h = X_train_legit.shape[1]
w = X_train_legit.shape[2]
triple=[np.zeros((batch_size, h, w,3)) for i in range(3)]

for l in range(0,trials):
    number_batches = int(X_test_phish_non_ordered.shape[0]/batch_size)
    count = 0
    for i in range(0,number_batches):
        for j in range(0,batch_size):
            first_img = X_test_phish_non_ordered[i*batch_size+j,:]
            triple[0][j,:,:,:] = first_img
            first_img_label = int(y_test_phish_non_ordered[i*batch_size+j,:])


            # get pos image by finding the closest image.
            if pairwise_distance_idx[i*batch_size+j,1] == 1:
                #from phish_train 
                pos_img = X_train_phish[int(pairwise_distance_idx[i*batch_size+j,0])]
            else:
                pos_img = X_train_legit[int(pairwise_distance_idx[i*batch_size+j,0])]
            
            triple[1][j,:,:,:] = pos_img

            #get image for the thrid: negative from legit
            neg_img,label_neg = pick_neg_img(first_img_label,155)
            while check_if_same_category(first_img_label,label_neg) == 1:
                neg_img,label_neg = pick_neg_img(first_img_label,155)
            triple[2][j,:,:,:] = neg_img

        anchor_noise,anchor_adv = get_adv_example(triple,epsilon)
        for k in range(0,len(anchor_adv)):
            X_test_phish_adv[count,:] = anchor_adv[k,:]
            count = count + 1
    X_test_phish_adv_features = inside_model.predict(X_test_phish_adv)
    np.save(output_dir + 'X_test_phish_adv_closest_'+str(l),X_test_phish_adv_features)